In [3]:
import os
import talib
import logging
import pandas as pd
from datetime import datetime

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

root_path = "C:\\Users\\Fang\\Documents\\stock_data\\basic_data"
# root_path = "C:\\Users\\Flora\\Documents\\股票分析\\stock_data\\basic_data"

In [4]:
df_list = []
start_dt = '2008-03-01'
end_dt = '2020-07-24'
bband_slope_change_criteria = 0.5
bband_width_change_criteria = 0.25
prev_bband_width_criteria = 5
for file in os.listdir(root_path):
    if '.csv' not in file:
        continue
    logging.info(file.replace('.csv', ''))
    
    # load stock price data
    file_path = root_path + os.sep + file
    try:
        stock_df = pd.read_csv(file_path, index_col=None, header=0, dtype={'股票代號':str}, engine='python', encoding='utf-8').sort_values('日期')
    except:
        stock_df = pd.read_csv(file_path, index_col=None, header=0, dtype={'股票代號':str}, engine='python').sort_values('日期')
    stock_df.loc[:, '日期'] = pd.to_datetime(stock_df['日期'].str.replace('＊', ''))
    stock_df = stock_df[(stock_df['日期'] >= start_dt) & (stock_df['日期'] <= end_dt)]
    if len(stock_df) < 60:
        continue
    stock_df = stock_df.sort_values('日期')
    stock_df['收盤價'].fillna(method='ffill', inplace=True)
    
    # calculate BBands and some features
    closed = stock_df['收盤價'].values
    upper,middle,lower = talib.BBANDS(closed,20, 2, 2, matype=talib.MA_Type.SMA)
    previous_upper = pd.Series(upper).shift(periods=1).to_numpy()
    previous_lower = pd.Series(lower).shift(periods=1).to_numpy()
    upper_change = pd.Series((upper - previous_upper) / previous_upper*100)
    lower_change = (lower - previous_lower) / previous_lower*100
    bband_width =  pd.Series((upper-lower) / middle*100)
    bband_width_ma = pd.Series(talib.SMA(bband_width, 5))
    
    # add BBands data to stock price dataframe
    stock_df = stock_df.reset_index().drop('index', axis=1)
    stock_df['隔日最高價'] = stock_df['最高價'].shift(periods=-1)
    stock_df['隔日收盤價'] = stock_df['收盤價'].shift(periods=-1)
    stock_df['bband_width'] = bband_width
    stock_df['prev_bband_width'] = stock_df['bband_width'].shift(periods=1)
    stock_df['bband_slope'] = upper_change
    stock_df['prev_bband_slope'] = stock_df['bband_slope'].shift(periods=1)
    stock_df['bband_slope_change'] = abs(stock_df['bband_slope'] - stock_df['prev_bband_slope']) / (abs(stock_df['prev_bband_slope'] + 0.00000001))
    stock_df['bband_width_change'] = (stock_df['bband_width'] - stock_df['prev_bband_width']) / (stock_df['prev_bband_width'] + 0.00000001)
    
    # filter data if meet the alert conditions
    filtered_df = stock_df[(stock_df['bband_slope_change']>bband_slope_change_criteria) 
                           & (stock_df['bband_width_change']>bband_width_change_criteria) 
                           & (stock_df['prev_bband_width']<prev_bband_width_criteria)]
    df_list.append(filtered_df)
alert_df = pd.concat(df_list, axis=0, ignore_index=True, sort=False)

2020-07-25 11:21:09,741 [INFO] 0015
2020-07-25 11:21:09,830 [INFO] 0050
2020-07-25 11:21:09,876 [INFO] 0051
2020-07-25 11:21:09,925 [INFO] 0052
2020-07-25 11:21:09,976 [INFO] 0053
2020-07-25 11:21:10,022 [INFO] 0054
2020-07-25 11:21:10,070 [INFO] 0055
2020-07-25 11:21:10,132 [INFO] 0056
2020-07-25 11:21:10,199 [INFO] 0057
2020-07-25 11:21:10,265 [INFO] 0058
2020-07-25 11:21:10,311 [INFO] 0059
2020-07-25 11:21:10,357 [INFO] 0060
2020-07-25 11:21:10,395 [INFO] 0061
2020-07-25 11:21:10,439 [INFO] 0080
2020-07-25 11:21:10,475 [INFO] 0081
2020-07-25 11:21:10,509 [INFO] 1101
2020-07-25 11:21:10,557 [INFO] 1102
2020-07-25 11:21:10,613 [INFO] 1103
2020-07-25 11:21:10,660 [INFO] 1104
2020-07-25 11:21:10,722 [INFO] 1108
2020-07-25 11:21:10,773 [INFO] 1109
2020-07-25 11:21:10,824 [INFO] 1110
2020-07-25 11:21:10,878 [INFO] 1201
2020-07-25 11:21:10,929 [INFO] 1203
2020-07-25 11:21:11,000 [INFO] 1210
2020-07-25 11:21:11,080 [INFO] 1213
2020-07-25 11:21:11,151 [INFO] 1215
2020-07-25 11:21:11,205 [INF

2020-07-25 11:21:21,354 [INFO] 1762
2020-07-25 11:21:21,398 [INFO] 1773
2020-07-25 11:21:21,448 [INFO] 1776
2020-07-25 11:21:21,479 [INFO] 1777
2020-07-25 11:21:21,535 [INFO] 1781
2020-07-25 11:21:21,584 [INFO] 1783
2020-07-25 11:21:21,628 [INFO] 1784
2020-07-25 11:21:21,678 [INFO] 1785
2020-07-25 11:21:21,726 [INFO] 1786
2020-07-25 11:21:21,762 [INFO] 1787
2020-07-25 11:21:21,809 [INFO] 1788
2020-07-25 11:21:21,871 [INFO] 1789
2020-07-25 11:21:21,913 [INFO] 1795
2020-07-25 11:21:21,960 [INFO] 1796
2020-07-25 11:21:21,986 [INFO] 1799
2020-07-25 11:21:22,034 [INFO] 1802
2020-07-25 11:21:22,092 [INFO] 1805
2020-07-25 11:21:22,139 [INFO] 1806
2020-07-25 11:21:22,189 [INFO] 1808
2020-07-25 11:21:22,239 [INFO] 1809
2020-07-25 11:21:22,289 [INFO] 1810
2020-07-25 11:21:22,338 [INFO] 1813
2020-07-25 11:21:22,392 [INFO] 1815
2020-07-25 11:21:22,451 [INFO] 1817
2020-07-25 11:21:22,488 [INFO] 1902
2020-07-25 11:21:22,536 [INFO] 1903
2020-07-25 11:21:22,584 [INFO] 1904
2020-07-25 11:21:22,643 [INF

2020-07-25 11:21:32,714 [INFO] 2468
2020-07-25 11:21:32,765 [INFO] 2471
2020-07-25 11:21:32,820 [INFO] 2472
2020-07-25 11:21:32,868 [INFO] 2473
2020-07-25 11:21:32,902 [INFO] 2474
2020-07-25 11:21:32,967 [INFO] 2475
2020-07-25 11:21:33,017 [INFO] 2476
2020-07-25 11:21:33,078 [INFO] 2477
2020-07-25 11:21:33,127 [INFO] 2478
2020-07-25 11:21:33,176 [INFO] 2480
2020-07-25 11:21:33,223 [INFO] 2481
2020-07-25 11:21:33,273 [INFO] 2482
2020-07-25 11:21:33,322 [INFO] 2483
2020-07-25 11:21:33,369 [INFO] 2484
2020-07-25 11:21:33,419 [INFO] 2485
2020-07-25 11:21:33,492 [INFO] 2486
2020-07-25 11:21:33,539 [INFO] 2488
2020-07-25 11:21:33,589 [INFO] 2489
2020-07-25 11:21:33,643 [INFO] 2491
2020-07-25 11:21:33,703 [INFO] 2492
2020-07-25 11:21:33,772 [INFO] 2493
2020-07-25 11:21:33,825 [INFO] 2494
2020-07-25 11:21:33,849 [INFO] 2495
2020-07-25 11:21:33,917 [INFO] 2496
2020-07-25 11:21:33,976 [INFO] 2497
2020-07-25 11:21:34,025 [INFO] 2498
2020-07-25 11:21:34,075 [INFO] 2499
2020-07-25 11:21:34,128 [INF

2020-07-25 11:21:43,966 [INFO] 3071
2020-07-25 11:21:44,012 [INFO] 3073
2020-07-25 11:21:44,070 [INFO] 3078
2020-07-25 11:21:44,127 [INFO] 3079
2020-07-25 11:21:44,141 [INFO] 3080
2020-07-25 11:21:44,174 [INFO] 3081
2020-07-25 11:21:44,210 [INFO] 3083
2020-07-25 11:21:44,263 [INFO] 3085
2020-07-25 11:21:44,311 [INFO] 3086
2020-07-25 11:21:44,363 [INFO] 3087
2020-07-25 11:21:44,397 [INFO] 3088
2020-07-25 11:21:44,461 [INFO] 3089
2020-07-25 11:21:44,510 [INFO] 3090
2020-07-25 11:21:44,563 [INFO] 3092
2020-07-25 11:21:44,618 [INFO] 3093
2020-07-25 11:21:44,675 [INFO] 3094
2020-07-25 11:21:44,724 [INFO] 3095
2020-07-25 11:21:44,774 [INFO] 3099
2020-07-25 11:21:44,799 [INFO] 3105
2020-07-25 11:21:44,865 [INFO] 3114
2020-07-25 11:21:44,918 [INFO] 3115
2020-07-25 11:21:44,968 [INFO] 3118
2020-07-25 11:21:45,018 [INFO] 3122
2020-07-25 11:21:45,056 [INFO] 3126
2020-07-25 11:21:45,097 [INFO] 3128
2020-07-25 11:21:45,145 [INFO] 3130
2020-07-25 11:21:45,198 [INFO] 3131
2020-07-25 11:21:45,243 [INF

2020-07-25 11:21:55,147 [INFO] 3617
2020-07-25 11:21:55,191 [INFO] 3622
2020-07-25 11:21:55,240 [INFO] 3623
2020-07-25 11:21:55,285 [INFO] 3624
2020-07-25 11:21:55,334 [INFO] 3625
2020-07-25 11:21:55,379 [INFO] 3628
2020-07-25 11:21:55,423 [INFO] 3629
2020-07-25 11:21:55,465 [INFO] 3630
2020-07-25 11:21:55,511 [INFO] 3631
2020-07-25 11:21:55,574 [INFO] 3632
2020-07-25 11:21:55,634 [INFO] 3638
2020-07-25 11:21:55,665 [INFO] 3642
2020-07-25 11:21:55,706 [INFO] 3645
2020-07-25 11:21:55,748 [INFO] 3646
2020-07-25 11:21:55,792 [INFO] 3652
2020-07-25 11:21:55,837 [INFO] 3653
2020-07-25 11:21:55,887 [INFO] 3658
2020-07-25 11:21:55,919 [INFO] 3661
2020-07-25 11:21:55,955 [INFO] 3662
2020-07-25 11:21:55,987 [INFO] 3663
2020-07-25 11:21:56,027 [INFO] 3664
2020-07-25 11:21:56,072 [INFO] 3665
2020-07-25 11:21:56,114 [INFO] 3666
2020-07-25 11:21:56,149 [INFO] 3669
2020-07-25 11:21:56,197 [INFO] 3672
2020-07-25 11:21:56,233 [INFO] 3673
2020-07-25 11:21:56,275 [INFO] 3675
2020-07-25 11:21:56,317 [INF

2020-07-25 11:22:04,772 [INFO] 4934
2020-07-25 11:22:04,815 [INFO] 4935
2020-07-25 11:22:04,861 [INFO] 4938
2020-07-25 11:22:04,920 [INFO] 4939
2020-07-25 11:22:04,960 [INFO] 4942
2020-07-25 11:22:05,010 [INFO] 4943
2020-07-25 11:22:05,041 [INFO] 4944
2020-07-25 11:22:05,082 [INFO] 4946
2020-07-25 11:22:05,126 [INFO] 4947
2020-07-25 11:22:05,175 [INFO] 4950
2020-07-25 11:22:05,214 [INFO] 4952
2020-07-25 11:22:05,254 [INFO] 4953
2020-07-25 11:22:05,295 [INFO] 4956
2020-07-25 11:22:05,339 [INFO] 4958
2020-07-25 11:22:05,388 [INFO] 4960
2020-07-25 11:22:05,431 [INFO] 4961
2020-07-25 11:22:05,455 [INFO] 4965
2020-07-25 11:22:05,497 [INFO] 4966
2020-07-25 11:22:05,541 [INFO] 4967
2020-07-25 11:22:05,565 [INFO] 4968
2020-07-25 11:22:05,598 [INFO] 4971
2020-07-25 11:22:05,634 [INFO] 4972
2020-07-25 11:22:05,671 [INFO] 4973
2020-07-25 11:22:05,710 [INFO] 4974
2020-07-25 11:22:05,758 [INFO] 4976
2020-07-25 11:22:05,800 [INFO] 4977
2020-07-25 11:22:05,836 [INFO] 4979
2020-07-25 11:22:05,881 [INF

2020-07-25 11:22:14,491 [INFO] 6021
2020-07-25 11:22:14,539 [INFO] 6022
2020-07-25 11:22:14,584 [INFO] 6023
2020-07-25 11:22:14,641 [INFO] 6024
2020-07-25 11:22:14,695 [INFO] 6026
2020-07-25 11:22:14,726 [INFO] 6101
2020-07-25 11:22:14,773 [INFO] 6103
2020-07-25 11:22:14,830 [INFO] 6104
2020-07-25 11:22:14,878 [INFO] 6105
2020-07-25 11:22:14,923 [INFO] 6107
2020-07-25 11:22:14,965 [INFO] 6108
2020-07-25 11:22:15,015 [INFO] 6109
2020-07-25 11:22:15,073 [INFO] 6110
2020-07-25 11:22:15,097 [INFO] 6111
2020-07-25 11:22:15,148 [INFO] 6112
2020-07-25 11:22:15,213 [INFO] 6113
2020-07-25 11:22:15,261 [INFO] 6114
2020-07-25 11:22:15,309 [INFO] 6115
2020-07-25 11:22:15,367 [INFO] 6116
2020-07-25 11:22:15,419 [INFO] 6117
2020-07-25 11:22:15,470 [INFO] 6118
2020-07-25 11:22:15,519 [INFO] 6119
2020-07-25 11:22:15,551 [INFO] 6120
2020-07-25 11:22:15,600 [INFO] 6121
2020-07-25 11:22:15,671 [INFO] 6122
2020-07-25 11:22:15,720 [INFO] 6123
2020-07-25 11:22:15,781 [INFO] 6124
2020-07-25 11:22:15,836 [INF

2020-07-25 11:22:25,480 [INFO] 6512
2020-07-25 11:22:25,512 [INFO] 6514
2020-07-25 11:22:25,545 [INFO] 6516
2020-07-25 11:22:25,569 [INFO] 6523
2020-07-25 11:22:25,601 [INFO] 6525
2020-07-25 11:22:25,633 [INFO] 6527
2020-07-25 11:22:25,657 [INFO] 6530
2020-07-25 11:22:25,685 [INFO] 6531
2020-07-25 11:22:25,717 [INFO] 6532
2020-07-25 11:22:25,747 [INFO] 6533
2020-07-25 11:22:25,777 [INFO] 6535
2020-07-25 11:22:25,808 [INFO] 6538
2020-07-25 11:22:25,840 [INFO] 6541
2020-07-25 11:22:25,866 [INFO] 6542
2020-07-25 11:22:25,897 [INFO] 6547
2020-07-25 11:22:25,923 [INFO] 6548
2020-07-25 11:22:25,954 [INFO] 6552
2020-07-25 11:22:25,985 [INFO] 6554
2020-07-25 11:22:26,009 [INFO] 6556
2020-07-25 11:22:26,036 [INFO] 6558
2020-07-25 11:22:26,059 [INFO] 6560
2020-07-25 11:22:26,090 [INFO] 6561
2020-07-25 11:22:26,115 [INFO] 6568
2020-07-25 11:22:26,145 [INFO] 6569
2020-07-25 11:22:26,182 [INFO] 6570
2020-07-25 11:22:26,209 [INFO] 6573
2020-07-25 11:22:26,237 [INFO] 6574
2020-07-25 11:22:26,266 [INF

2020-07-25 11:22:34,401 [INFO] 8488
2020-07-25 11:22:34,431 [INFO] 8489
2020-07-25 11:22:34,460 [INFO] 8497
2020-07-25 11:22:34,487 [INFO] 8499
2020-07-25 11:22:34,515 [INFO] 8705
2020-07-25 11:22:34,551 [INFO] 8905
2020-07-25 11:22:34,604 [INFO] 8906
2020-07-25 11:22:34,657 [INFO] 8908
2020-07-25 11:22:34,706 [INFO] 8913
2020-07-25 11:22:34,763 [INFO] 8916
2020-07-25 11:22:34,814 [INFO] 8917
2020-07-25 11:22:34,861 [INFO] 8921
2020-07-25 11:22:34,909 [INFO] 8923
2020-07-25 11:22:34,957 [INFO] 8924
2020-07-25 11:22:35,007 [INFO] 8925
2020-07-25 11:22:35,053 [INFO] 8926
2020-07-25 11:22:35,102 [INFO] 8927
2020-07-25 11:22:35,152 [INFO] 8928
2020-07-25 11:22:35,209 [INFO] 8929
2020-07-25 11:22:35,258 [INFO] 8930
2020-07-25 11:22:35,316 [INFO] 8931
2020-07-25 11:22:35,395 [INFO] 8932
2020-07-25 11:22:35,443 [INFO] 8933
2020-07-25 11:22:35,500 [INFO] 8934
2020-07-25 11:22:35,555 [INFO] 8935
2020-07-25 11:22:35,607 [INFO] 8936
2020-07-25 11:22:35,662 [INFO] 8937
2020-07-25 11:22:35,741 [INF

In [5]:
alert_df[alert_df['日期']=='2020-07-24'].sort_values(['日期', '成交金額'])

,股票代號,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,漲跌幅,成交筆數,隔日最高價,隔日收盤價,bband_width,prev_bband_width,bband_slope,prev_bband_slope,bband_slope_change,bband_width_change
17988,4406,2020-07-24,0.0,0.0,-1.00,-1.00,-1.00,-1.00,-1.00,-inf,0.0,NaN,NaN,102.773141,3.643975,40.338817,-0.271394,149.635682,27.203578
27689,8401,2020-07-24,29000.0,326300.0,11.30,11.45,11.20,11.20,-0.15,-1.32,8.0,NaN,NaN,3.847472,3.073602,0.272040,-0.084106,4.234503,0.251779
21333,5489,2020-07-24,84576.0,2558180.0,30.50,30.50,30.05,30.20,-0.30,-0.98,50.0,NaN,NaN,4.887855,3.760875,0.336571,0.036331,8.263892,0.299659
29072,9905,2020-07-24,217974.0,4574650.0,21.20,21.20,20.85,20.85,-0.35,-1.65,111.0,NaN,NaN,6.273003,4.764158,0.495046,0.292784,0.690822,0.316708
10891,2608,2020-07-24,348800.0,14030659.0,40.90,41.00,39.05,39.80,-1.15,-2.81,217.0,NaN,NaN,3.602808,2.639689,0.364928,0.003756,96.162159,0.364861
1312,1304,2020-07-24,2475999.0,29404170.0,12.10,12.10,11.75,11.80,-0.30,-2.48,886.0,NaN,NaN,6.448744,4.734414,0.573530,0.043153,12.290625,0.362100
12510,2867,2020-07-24,3445043.0,33910278.0,9.94,9.94,9.77,9.80,-0.18,-1.80,938.0,NaN,NaN,3.117678,2.344536,0.257077,-0.006295,41.836967,0.329763
29762,9933,2020-07-24,2671897.0,97996276.0,36.55,37.00,36.45,36.50,-0.00,-0.00,1559.0,NaN,NaN,6.101516,3.664839,0.818087,-0.215638,4.793798,0.664880
15618,3380,2020-07-24,8987949.0,237390313.0,27.80,27.80,25.55,25.65,-2.10,-7.57,4420.0,NaN,NaN,7.776086,4.257117,1.438927,-0.973946,2.477420,0.826609
6175,2002,2020-07-24,21544000.0,435800799.0,20.20,20.35,20.15,20.15,-0.00,-0.00,6777.0,NaN,NaN,3.466425,2.406167,0.378439,-0.027713,14.655516,0.440642


In [23]:
alert_df[(alert_df['漲跌幅']>0) & (alert_df['prev_bband_slope']>0)].shape

(10723, 18)

In [28]:
alert_df[(alert_df['漲跌幅']>0) & (alert_df['收盤價'] < alert_df['隔日收盤價']) & (alert_df['prev_bband_slope']>0)]

,股票代號,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,漲跌幅,成交筆數,隔日最高價,隔日收盤價,bband_width,prev_bband_width,bband_slope,prev_bband_slope,bband_slope_change,bband_width_change
3,0015,2010-12-01,96000.0,7.222100e+05,7.49,7.55,7.49,7.55,0.08,1.07,23.0,7.65,7.62,2.842085,2.173721,0.432351,0.150487,1.873005,0.307474
13,0015,2013-11-27,123000.0,1.093660e+06,8.85,8.93,8.85,8.93,0.08,0.90,16.0,9.04,9.00,3.985536,3.159316,0.580220,0.376909,0.539418,0.261519
16,0050,2010-09-13,18145999.0,1.019516e+09,55.55,56.80,55.55,56.80,1.55,2.81,3161.0,57.00,57.00,5.571667,4.264479,0.806090,0.104194,6.736436,0.306530
19,0050,2012-09-14,35619400.0,1.967238e+09,54.80,55.60,54.80,55.50,1.35,2.49,6519.0,55.80,55.60,5.439646,3.701005,1.095894,0.336424,2.257476,0.469775
26,0050,2015-04-24,26527170.0,1.917092e+09,71.70,72.85,71.70,72.20,0.70,0.98,4260.0,73.30,72.75,5.055846,3.738149,0.788205,0.477038,0.652290,0.352500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30267,9960,2014-05-09,147000.0,2.607650e+06,17.50,18.00,17.45,17.65,0.20,1.15,77.0,17.85,17.70,5.622192,4.174650,0.962197,0.463755,1.074795,0.346746
30274,9960,2018-04-03,9016.0,2.976780e+05,33.30,33.35,32.35,33.35,0.10,0.30,25.0,34.50,33.75,6.101391,4.787813,0.854006,0.174468,3.894915,0.274359
30297,9962,2017-02-15,1387294.0,1.453614e+07,10.30,10.75,10.20,10.60,0.30,2.91,343.0,11.20,10.70,6.705433,4.619144,1.347010,0.504199,1.671584,0.451661
30301,9962,2018-02-23,238230.0,2.536411e+06,10.55,10.85,10.45,10.75,0.25,2.38,89.0,11.00,10.80,4.510015,3.341233,0.720058,0.122934,4.857263,0.349806
